In [2]:
import pandas as pd
import numpy as np

training_csv = pd.read_csv('strain0.csv',index_col=0)
testing_csv = pd.read_csv('stest0.csv',index_col=0)
result_csv = pd.read_csv('sample_submission_2.csv')

x_train = training_csv.drop(training_csv.columns[range(149, 213)], axis=1)
x_train_data = x_train.values

weight_intraday = training_csv.iloc[:,211]
weight_intraday_data = weight_intraday.values
weight_daily = training_csv.iloc[:212]
weight_daily_data = weight_daily.values

testing_data = testing_csv.values

In [18]:
def WMAE(preds,dtrain): #preds是结果（概率值），dtrain是个带label的DMatrix
    labels=dtrain.get_label()     #提取label
    weights = dtrain.get_weight()  #提取weight
    score = np.mean(weights*np.abs(labels - preds))
    return 'WMAE:',score

In [ ]:
import xgboost as xgb  #xgboost
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBRegressor  #xgb的sklearn包

def WMAE(preds,dtrain): #preds是结果（概率值），dtrain是个带label的DMatrix
    labels=dtrain.get_label()     #提取label
    weights = dtrain.get_weight()  #提取weight
    score = np.mean(weights*np.abs(labels - preds))
    return 'WMAE:',float(score)

xgbpredict = np.zeros((120000, 62))

for i in range(0,62): # From 1 to 62

    model_parameters_inday = {
                        'max_depth': 10, 
                        'eta': 0.01, 
                        'gamma': 0, 
                        'lambda': 10, 
                        'alpha': 10, 
                        'min_child_weight':2, 
                        'colsample_bytree':0.95, 
                        'subsample':0.95,
                        #'eval_metric':'mae',
                        'seed':27
                       }
    model_parameters_daily = {
                        'max_depth': 10, 
                        'eta': 0.01, 
                        'gamma': 0, 
                        'lambda': 10, 
                        'alpha': 10, 
                        'min_child_weight':2, 
                        'colsample_bytree':0.95, 
                        'subsample':0.95,
                        #'eval_metric':'mae',
                        'seed':27
                        }
    
    
    if i == 60:
        name_of_column = 'Ret_PlusOne'
        name_of_weight = 'Weight_Daily'
        model_parameters = model_parameters_daily
    elif i == 61:
        name_of_column = 'Ret_PlusTwo'
        name_of_weight = 'Weight_Daily'
        model_parameters = model_parameters_daily
    else:
        name_of_column = 'Ret_'+str(i+121)
        name_of_weight = 'Weight_Intraday'
        model_parameters = model_parameters_inday
        
    y_train_data = training_csv[name_of_column].values
    weight_data = training_csv[name_of_weight].values
    data_train = xgb.DMatrix(x_train_data, label=y_train_data, weight=weight_data)
    data_test = xgb.DMatrix(testing_data)
    
    number_of_rounds = 1100
    
    watchlist = [(data_train, 'train')]
    bst = xgb.train(model_parameters, data_train, number_of_rounds,watchlist,early_stopping_rounds=10)
    
    predictions = bst.predict(data_test)
    xgbpredict[:,i]=predictions

        

resultnum=np.zeros(7440000)
for j in range(0,120000):
    resultnum[62*j:62+62*j]=xgbpredict[j,:]
result_csv.iloc[:,1]=resultnum
result_csv.to_csv('result-xgboost-med.csv',index=False)

[0]	train-rmse:0.49503
Will train until train-rmse hasn't improved in 10 rounds.
[1]	train-rmse:0.490079
[2]	train-rmse:0.485179
[3]	train-rmse:0.480327
[4]	train-rmse:0.475524
[5]	train-rmse:0.470768
[6]	train-rmse:0.466061
[7]	train-rmse:0.4614
[8]	train-rmse:0.456786
[9]	train-rmse:0.452218
[10]	train-rmse:0.447696
[11]	train-rmse:0.443219
[12]	train-rmse:0.438787
[13]	train-rmse:0.434399
[14]	train-rmse:0.430055
[15]	train-rmse:0.425755
[16]	train-rmse:0.421497
[17]	train-rmse:0.417282
[18]	train-rmse:0.413109
[19]	train-rmse:0.408978
[20]	train-rmse:0.404889
[21]	train-rmse:0.40084
[22]	train-rmse:0.396831
[23]	train-rmse:0.392863
[24]	train-rmse:0.388934
[25]	train-rmse:0.385045
[26]	train-rmse:0.381195
[27]	train-rmse:0.377383
[28]	train-rmse:0.373609
[29]	train-rmse:0.369873
[30]	train-rmse:0.366174
[31]	train-rmse:0.362512
[32]	train-rmse:0.358887
[33]	train-rmse:0.355298
[34]	train-rmse:0.351745
[35]	train-rmse:0.348228
[36]	train-rmse:0.344746
[37]	train-rmse:0.341298
[38]	t

[319]	train-rmse:0.020068
[320]	train-rmse:0.019868
[321]	train-rmse:0.01967
[322]	train-rmse:0.019473
[323]	train-rmse:0.019279
[324]	train-rmse:0.019086
[325]	train-rmse:0.018895
[326]	train-rmse:0.018707
[327]	train-rmse:0.01852
[328]	train-rmse:0.018335
[329]	train-rmse:0.018152
[330]	train-rmse:0.01797
[331]	train-rmse:0.017791
[332]	train-rmse:0.017613
[333]	train-rmse:0.017437
[334]	train-rmse:0.017263
[335]	train-rmse:0.017091
[336]	train-rmse:0.01692
[337]	train-rmse:0.016751
[338]	train-rmse:0.016584
[339]	train-rmse:0.016418
[340]	train-rmse:0.016254
[341]	train-rmse:0.016092
[342]	train-rmse:0.015931
[343]	train-rmse:0.015772
[344]	train-rmse:0.015615
[345]	train-rmse:0.015459
[346]	train-rmse:0.015305
[347]	train-rmse:0.015152
[348]	train-rmse:0.015001
[349]	train-rmse:0.014851
[350]	train-rmse:0.014703
[351]	train-rmse:0.014556
[352]	train-rmse:0.01441
[353]	train-rmse:0.014267
[354]	train-rmse:0.014124
[355]	train-rmse:0.013983
[356]	train-rmse:0.013844
[357]	train-rmse:

[636]	train-rmse:0.00095
[637]	train-rmse:0.000943
[638]	train-rmse:0.000935
[639]	train-rmse:0.000928
[640]	train-rmse:0.000921
[641]	train-rmse:0.000914
[642]	train-rmse:0.000906
[643]	train-rmse:0.0009
[644]	train-rmse:0.000893
[645]	train-rmse:0.000886
[646]	train-rmse:0.000879
[647]	train-rmse:0.000872
[648]	train-rmse:0.000866
[649]	train-rmse:0.000859
[650]	train-rmse:0.000853
[651]	train-rmse:0.000847
[652]	train-rmse:0.00084
[653]	train-rmse:0.000834
[654]	train-rmse:0.000828
[655]	train-rmse:0.000822
[656]	train-rmse:0.000816
[657]	train-rmse:0.00081
[658]	train-rmse:0.000804
[659]	train-rmse:0.000798
[660]	train-rmse:0.000792
[661]	train-rmse:0.000787
[662]	train-rmse:0.000781
[663]	train-rmse:0.000776
[664]	train-rmse:0.00077
[665]	train-rmse:0.000765
[666]	train-rmse:0.00076
[667]	train-rmse:0.000754
[668]	train-rmse:0.000749
[669]	train-rmse:0.000744
[670]	train-rmse:0.000739
[671]	train-rmse:0.000734
[672]	train-rmse:0.000729
[673]	train-rmse:0.000724
[674]	train-rmse:0.

[953]	train-rmse:0.000328
[954]	train-rmse:0.000328
[955]	train-rmse:0.000327
[956]	train-rmse:0.000327
[957]	train-rmse:0.000327
[958]	train-rmse:0.000327
[959]	train-rmse:0.000326
[960]	train-rmse:0.000326
[961]	train-rmse:0.000326
[962]	train-rmse:0.000325
[963]	train-rmse:0.000325
[964]	train-rmse:0.000324
[965]	train-rmse:0.000324
[966]	train-rmse:0.000324
[967]	train-rmse:0.000323
[968]	train-rmse:0.000323
[969]	train-rmse:0.000323
[970]	train-rmse:0.000323
[971]	train-rmse:0.000322
[972]	train-rmse:0.000322
[973]	train-rmse:0.000322
[974]	train-rmse:0.000321
[975]	train-rmse:0.000321
[976]	train-rmse:0.000321
[977]	train-rmse:0.00032
[978]	train-rmse:0.00032
[979]	train-rmse:0.00032
[980]	train-rmse:0.000319
[981]	train-rmse:0.000319
[982]	train-rmse:0.000319
[983]	train-rmse:0.000318
[984]	train-rmse:0.000318
[985]	train-rmse:0.000317
[986]	train-rmse:0.000317
[987]	train-rmse:0.000317
[988]	train-rmse:0.000316
[989]	train-rmse:0.000316
[990]	train-rmse:0.000316
[991]	train-rms

[168]	train-rmse:0.091486
[169]	train-rmse:0.090572
[170]	train-rmse:0.089666
[171]	train-rmse:0.088769
[172]	train-rmse:0.087882
[173]	train-rmse:0.087003
[174]	train-rmse:0.086133
[175]	train-rmse:0.085272
[176]	train-rmse:0.084419
[177]	train-rmse:0.083575
[178]	train-rmse:0.082739
[179]	train-rmse:0.081912
[180]	train-rmse:0.081093
[181]	train-rmse:0.080282
[182]	train-rmse:0.079479
[183]	train-rmse:0.078685
[184]	train-rmse:0.077898
[185]	train-rmse:0.077119
[186]	train-rmse:0.076348
[187]	train-rmse:0.075584
[188]	train-rmse:0.074829
[189]	train-rmse:0.07408
[190]	train-rmse:0.07334
[191]	train-rmse:0.072606
[192]	train-rmse:0.07188
[193]	train-rmse:0.071162
[194]	train-rmse:0.07045
[195]	train-rmse:0.069746
[196]	train-rmse:0.069048
[197]	train-rmse:0.068358
[198]	train-rmse:0.067674
[199]	train-rmse:0.066998
[200]	train-rmse:0.066328
[201]	train-rmse:0.065665
[202]	train-rmse:0.065008
[203]	train-rmse:0.064358
[204]	train-rmse:0.063715
[205]	train-rmse:0.063078
[206]	train-rmse

...

[34]	train-rmse:0.351689
[35]	train-rmse:0.348173
[36]	train-rmse:0.344691
[37]	train-rmse:0.341244
[38]	train-rmse:0.337832
[39]	train-rmse:0.334453
[40]	train-rmse:0.331109
[41]	train-rmse:0.327798
[42]	train-rmse:0.32452
[43]	train-rmse:0.321275
[44]	train-rmse:0.318062
[45]	train-rmse:0.314881
[46]	train-rmse:0.311733
[47]	train-rmse:0.308615
[48]	train-rmse:0.305529
[49]	train-rmse:0.302474
[50]	train-rmse:0.299449
[51]	train-rmse:0.296455
[52]	train-rmse:0.29349
[53]	train-rmse:0.290555
[54]	train-rmse:0.28765
[55]	train-rmse:0.284773
[56]	train-rmse:0.281926
[57]	train-rmse:0.279106
[58]	train-rmse:0.276315
[59]	train-rmse:0.273552
[60]	train-rmse:0.270817
[61]	train-rmse:0.268109
[62]	train-rmse:0.265428
[63]	train-rmse:0.262773
[64]	train-rmse:0.260146
[65]	train-rmse:0.257544
[66]	train-rmse:0.254969
[67]	train-rmse:0.252419
[68]	train-rmse:0.249895
[69]	train-rmse:0.247396
[70]	train-rmse:0.244922
[71]	train-rmse:0.242473
[72]	train-rmse:0.240048
[73]	train-rmse:0.237648
[74

[354]	train-rmse:0.014131
[355]	train-rmse:0.01399
[356]	train-rmse:0.01385
[357]	train-rmse:0.013712
[358]	train-rmse:0.013576
[359]	train-rmse:0.01344
[360]	train-rmse:0.013306
[361]	train-rmse:0.013174
[362]	train-rmse:0.013042
[363]	train-rmse:0.012912
[364]	train-rmse:0.012784
[365]	train-rmse:0.012656
[366]	train-rmse:0.01253
[367]	train-rmse:0.012405
[368]	train-rmse:0.012282
[369]	train-rmse:0.012159
[370]	train-rmse:0.012038
[371]	train-rmse:0.011918
[372]	train-rmse:0.0118
[373]	train-rmse:0.011682
[374]	train-rmse:0.011566
[375]	train-rmse:0.011451
[376]	train-rmse:0.011337
[377]	train-rmse:0.011224
[378]	train-rmse:0.011112
[379]	train-rmse:0.011002
[380]	train-rmse:0.010892
[381]	train-rmse:0.010784
[382]	train-rmse:0.010676
[383]	train-rmse:0.01057
[384]	train-rmse:0.010465
[385]	train-rmse:0.010361
[386]	train-rmse:0.010258
[387]	train-rmse:0.010156
[388]	train-rmse:0.010055
[389]	train-rmse:0.009955
[390]	train-rmse:0.009856
[391]	train-rmse:0.009758
[392]	train-rmse:0.

[672]	train-rmse:0.000809
[673]	train-rmse:0.000804
[674]	train-rmse:0.0008
[675]	train-rmse:0.000795
[676]	train-rmse:0.000791
[677]	train-rmse:0.000787
[678]	train-rmse:0.000783
[679]	train-rmse:0.000778
[680]	train-rmse:0.000774
[681]	train-rmse:0.00077
[682]	train-rmse:0.000766
[683]	train-rmse:0.000762
[684]	train-rmse:0.000758
[685]	train-rmse:0.000754
[686]	train-rmse:0.00075
[687]	train-rmse:0.000746
[688]	train-rmse:0.000743
[689]	train-rmse:0.000739
[690]	train-rmse:0.000736
[691]	train-rmse:0.000732
[692]	train-rmse:0.000729
[693]	train-rmse:0.000725
[694]	train-rmse:0.000722
[695]	train-rmse:0.000718
[696]	train-rmse:0.000715
[697]	train-rmse:0.000711
[698]	train-rmse:0.000708
[699]	train-rmse:0.000705
[700]	train-rmse:0.000702
[701]	train-rmse:0.000699
[702]	train-rmse:0.000696
[703]	train-rmse:0.000693
[704]	train-rmse:0.00069
[705]	train-rmse:0.000687
[706]	train-rmse:0.000684
[707]	train-rmse:0.000682
[708]	train-rmse:0.000679
[709]	train-rmse:0.000676
[710]	train-rmse:

[989]	train-rmse:0.000414
[990]	train-rmse:0.000414
[991]	train-rmse:0.000413
[992]	train-rmse:0.000413
[993]	train-rmse:0.000412
[994]	train-rmse:0.000412
[995]	train-rmse:0.000412
[996]	train-rmse:0.000412
[997]	train-rmse:0.000411
[998]	train-rmse:0.000411
[999]	train-rmse:0.00041
[1000]	train-rmse:0.00041
[1001]	train-rmse:0.000409
[1002]	train-rmse:0.000409
[1003]	train-rmse:0.000408
[1004]	train-rmse:0.000408
[1005]	train-rmse:0.000408
[1006]	train-rmse:0.000407
[1007]	train-rmse:0.000407
[1008]	train-rmse:0.000407
[1009]	train-rmse:0.000407
[1010]	train-rmse:0.000406
[1011]	train-rmse:0.000406
[1012]	train-rmse:0.000406
[1013]	train-rmse:0.000405
[1014]	train-rmse:0.000405
[1015]	train-rmse:0.000404
[1016]	train-rmse:0.000404
[1017]	train-rmse:0.000404
[1018]	train-rmse:0.000404
[1019]	train-rmse:0.000403
[1020]	train-rmse:0.000403
[1021]	train-rmse:0.000403
[1022]	train-rmse:0.000402
[1023]	train-rmse:0.000402
[1024]	train-rmse:0.000402
[1025]	train-rmse:0.000401
[1026]	train-r

[204]	train-rmse:0.063712
[205]	train-rmse:0.063075
[206]	train-rmse:0.062445
[207]	train-rmse:0.06182
[208]	train-rmse:0.061202
[209]	train-rmse:0.06059
[210]	train-rmse:0.059984
[211]	train-rmse:0.059385
[212]	train-rmse:0.058791
[213]	train-rmse:0.058203
[214]	train-rmse:0.057621
[215]	train-rmse:0.057045
[216]	train-rmse:0.056475
[217]	train-rmse:0.05591
[218]	train-rmse:0.055351
[219]	train-rmse:0.054798
[220]	train-rmse:0.05425
[221]	train-rmse:0.053708
[222]	train-rmse:0.053171
[223]	train-rmse:0.052639
[224]	train-rmse:0.052113
[225]	train-rmse:0.051592
[226]	train-rmse:0.051076
[227]	train-rmse:0.050565
[228]	train-rmse:0.05006
[229]	train-rmse:0.049559
[230]	train-rmse:0.049064
[231]	train-rmse:0.048573
[232]	train-rmse:0.048088
[233]	train-rmse:0.047607
[234]	train-rmse:0.047131
[235]	train-rmse:0.04666
[236]	train-rmse:0.046193
[237]	train-rmse:0.045732
[238]	train-rmse:0.045274
[239]	train-rmse:0.044822
[240]	train-rmse:0.044374
[241]	train-rmse:0.04393
[242]	train-rmse:0.

In [38]:
import xgboost as xgb  #xgboost
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBRegressor  #xgb的sklearn包

xgbpredict = np.zeros((120000, 31))

for i in range(0,31): # From 1 to 62

    model_parameters_inday = {
                        'max_depth': 10, 
                        'eta': 0.01, 
                        'gamma': 0, 
                        'lambda': 10, 
                        'alpha': 10, 
                        'min_child_weight':2, 
                        'colsample_bytree':0.95, 
                        'subsample':0.95,
                        'eval_metric':'mae',
                        'seed':27
                       }
    model_parameters_daily = {
                        'max_depth': 10, 
                        'eta': 0.01, 
                        'gamma': 0, 
                        'lambda': 10, 
                        'alpha': 10, 
                        'min_child_weight':2, 
                        'colsample_bytree':0.95, 
                        'subsample':0.95,
                        'eval_metric':'mae',
                        'seed':27
                        }
    
    
    if i == 60:
        name_of_column = 'Ret_PlusOne'
        name_of_weight = 'Weight_Daily'
        model_parameters = model_parameters_daily
    elif i == 61:
        name_of_column = 'Ret_PlusTwo'
        name_of_weight = 'Weight_Daily'
        model_parameters = model_parameters_daily
    else:
        name_of_column = 'Ret_'+str(i+121)
        name_of_weight = 'Weight_Intraday'
        model_parameters = model_parameters_inday
        
    y_train_data = training_csv[name_of_column].values
    weight_data = training_csv[name_of_weight].values
    data_train = xgb.DMatrix(x_train_data, label=y_train_data, weight=weight_data)
    data_test = xgb.DMatrix(testing_data)
    
    number_of_rounds = 5000
    
    watchlist = [(data_train, 'train')]
    bst = xgb.train(model_parameters, data_train, number_of_rounds, watchlist, early_stopping_rounds=10)
    
    predictions = bst.predict(data_test)
    xgbpredict[:,i]=predictions

        

result1 = pd.DataFrame(xgbpredict)
result1.to_csv('result1.csv',index=False)

[0]	train-mae:0.495029
Will train until train-mae hasn't improved in 10 rounds.
[1]	train-mae:0.490078
[2]	train-mae:0.485177
[3]	train-mae:0.480326
[4]	train-mae:0.475522
[5]	train-mae:0.470767
[6]	train-mae:0.46606
[7]	train-mae:0.461399
[8]	train-mae:0.456785
[9]	train-mae:0.452217
[10]	train-mae:0.447695
[11]	train-mae:0.443218
[12]	train-mae:0.438786
[13]	train-mae:0.434398
[14]	train-mae:0.430054
[15]	train-mae:0.425753
[16]	train-mae:0.421496
[17]	train-mae:0.417281
[18]	train-mae:0.413108
[19]	train-mae:0.408977
[20]	train-mae:0.404887
[21]	train-mae:0.400838
[22]	train-mae:0.39683
[23]	train-mae:0.392862
[24]	train-mae:0.388933
[25]	train-mae:0.385044
[26]	train-mae:0.381193
[27]	train-mae:0.377381
[28]	train-mae:0.373608
[29]	train-mae:0.369871
[30]	train-mae:0.366173
[31]	train-mae:0.362511
[32]	train-mae:0.358886
[33]	train-mae:0.355297
[34]	train-mae:0.351744
[35]	train-mae:0.348227
[36]	train-mae:0.344744
[37]	train-mae:0.341297
[38]	train-mae:0.337884
[39]	train-mae:0.33

[332]	train-mae:0.0176
[333]	train-mae:0.017424
[334]	train-mae:0.01725
[335]	train-mae:0.017077
[336]	train-mae:0.016907
[337]	train-mae:0.016738
[338]	train-mae:0.01657
[339]	train-mae:0.016405
[340]	train-mae:0.016241
[341]	train-mae:0.016078
[342]	train-mae:0.015917
[343]	train-mae:0.015758
[344]	train-mae:0.015601
[345]	train-mae:0.015445
[346]	train-mae:0.01529
[347]	train-mae:0.015137
[348]	train-mae:0.014986
[349]	train-mae:0.014836
[350]	train-mae:0.014688
[351]	train-mae:0.014541
[352]	train-mae:0.014395
[353]	train-mae:0.014251
[354]	train-mae:0.014109
[355]	train-mae:0.013968
[356]	train-mae:0.013828
[357]	train-mae:0.01369
[358]	train-mae:0.013553
[359]	train-mae:0.013417
[360]	train-mae:0.013283
[361]	train-mae:0.01315
[362]	train-mae:0.013019
[363]	train-mae:0.012889
[364]	train-mae:0.01276
[365]	train-mae:0.012632
[366]	train-mae:0.012506
[367]	train-mae:0.012381
[368]	train-mae:0.012257
[369]	train-mae:0.012134
[370]	train-mae:0.012013
[371]	train-mae:0.011893
[372]	tr

[662]	train-mae:0.00068
[663]	train-mae:0.000674
[664]	train-mae:0.000668
[665]	train-mae:0.000663
[666]	train-mae:0.000658
[667]	train-mae:0.000652
[668]	train-mae:0.000647
[669]	train-mae:0.000642
[670]	train-mae:0.000637
[671]	train-mae:0.000631
[672]	train-mae:0.000626
[673]	train-mae:0.000621
[674]	train-mae:0.000617
[675]	train-mae:0.000612
[676]	train-mae:0.000607
[677]	train-mae:0.000602
[678]	train-mae:0.000598
[679]	train-mae:0.000593
[680]	train-mae:0.000589
[681]	train-mae:0.000584
[682]	train-mae:0.00058
[683]	train-mae:0.000575
[684]	train-mae:0.000571
[685]	train-mae:0.000567
[686]	train-mae:0.000562
[687]	train-mae:0.000558
[688]	train-mae:0.000554
[689]	train-mae:0.00055
[690]	train-mae:0.000547
[691]	train-mae:0.000543
[692]	train-mae:0.000539
[693]	train-mae:0.000535
[694]	train-mae:0.000531
[695]	train-mae:0.000527
[696]	train-mae:0.000524
[697]	train-mae:0.00052
[698]	train-mae:0.000517
[699]	train-mae:0.000513
[700]	train-mae:0.00051
[701]	train-mae:0.000506
[702]

[992]	train-mae:0.000235
[993]	train-mae:0.000235
[994]	train-mae:0.000235
[995]	train-mae:0.000234
[996]	train-mae:0.000234
[997]	train-mae:0.000234
[998]	train-mae:0.000233
[999]	train-mae:0.000233
[1000]	train-mae:0.000233
[1001]	train-mae:0.000233
[1002]	train-mae:0.000233
[1003]	train-mae:0.000233
[1004]	train-mae:0.000232
[1005]	train-mae:0.000232
[1006]	train-mae:0.000232
[1007]	train-mae:0.000232
[1008]	train-mae:0.000232
[1009]	train-mae:0.000231
[1010]	train-mae:0.000231
[1011]	train-mae:0.000231
[1012]	train-mae:0.000231
[1013]	train-mae:0.00023
[1014]	train-mae:0.00023
[1015]	train-mae:0.00023
[1016]	train-mae:0.00023
[1017]	train-mae:0.00023
[1018]	train-mae:0.000229
[1019]	train-mae:0.000229
[1020]	train-mae:0.000229
[1021]	train-mae:0.000229
[1022]	train-mae:0.000228
[1023]	train-mae:0.000228
[1024]	train-mae:0.000228
[1025]	train-mae:0.000228
[1026]	train-mae:0.000228
[1027]	train-mae:0.000227
[1028]	train-mae:0.000227
[1029]	train-mae:0.000227
[1030]	train-mae:0.000227

[1310]	train-mae:0.000181
[1311]	train-mae:0.000181
[1312]	train-mae:0.000181
[1313]	train-mae:0.00018
[1314]	train-mae:0.00018
[1315]	train-mae:0.00018
[1316]	train-mae:0.00018
[1317]	train-mae:0.00018
[1318]	train-mae:0.00018
[1319]	train-mae:0.000179
[1320]	train-mae:0.000179
[1321]	train-mae:0.000179
[1322]	train-mae:0.000179
[1323]	train-mae:0.000179
[1324]	train-mae:0.000179
[1325]	train-mae:0.000179
[1326]	train-mae:0.000178
[1327]	train-mae:0.000178
[1328]	train-mae:0.000178
[1329]	train-mae:0.000178
[1330]	train-mae:0.000178
[1331]	train-mae:0.000178
[1332]	train-mae:0.000178
[1333]	train-mae:0.000177
[1334]	train-mae:0.000177
[1335]	train-mae:0.000177
[1336]	train-mae:0.000177
[1337]	train-mae:0.000177
[1338]	train-mae:0.000177
[1339]	train-mae:0.000177
[1340]	train-mae:0.000176
[1341]	train-mae:0.000176
[1342]	train-mae:0.000176
[1343]	train-mae:0.000176
[1344]	train-mae:0.000176
[1345]	train-mae:0.000176
[1346]	train-mae:0.000176
[1347]	train-mae:0.000175
[1348]	train-mae:0

...

[148]	train-mae:0.111844
[149]	train-mae:0.110725
[150]	train-mae:0.109618
[151]	train-mae:0.108522
[152]	train-mae:0.107437
[153]	train-mae:0.106362
[154]	train-mae:0.105299
[155]	train-mae:0.104246
[156]	train-mae:0.103203
[157]	train-mae:0.102171
[158]	train-mae:0.10115
[159]	train-mae:0.100138
[160]	train-mae:0.099137
[161]	train-mae:0.098145
[162]	train-mae:0.097164
[163]	train-mae:0.096192
[164]	train-mae:0.09523
[165]	train-mae:0.094278
[166]	train-mae:0.093335
[167]	train-mae:0.092402
[168]	train-mae:0.091478
[169]	train-mae:0.090563
[170]	train-mae:0.089657
[171]	train-mae:0.088761
[172]	train-mae:0.087873
[173]	train-mae:0.086995
[174]	train-mae:0.086125
[175]	train-mae:0.085263
[176]	train-mae:0.084411
[177]	train-mae:0.083567
[178]	train-mae:0.082731
[179]	train-mae:0.081904
[180]	train-mae:0.081085
[181]	train-mae:0.080274
[182]	train-mae:0.079471
[183]	train-mae:0.078676
[184]	train-mae:0.07789
[185]	train-mae:0.077111
[186]	train-mae:0.07634
[187]	train-mae:0.075576
[188

[478]	train-mae:0.004057
[479]	train-mae:0.004017
[480]	train-mae:0.003977
[481]	train-mae:0.003937
[482]	train-mae:0.003898
[483]	train-mae:0.003859
[484]	train-mae:0.00382
[485]	train-mae:0.003782
[486]	train-mae:0.003744
[487]	train-mae:0.003707
[488]	train-mae:0.00367
[489]	train-mae:0.003633
[490]	train-mae:0.003597
[491]	train-mae:0.003561
[492]	train-mae:0.003525
[493]	train-mae:0.00349
[494]	train-mae:0.003455
[495]	train-mae:0.00342
[496]	train-mae:0.003386
[497]	train-mae:0.003352
[498]	train-mae:0.003319
[499]	train-mae:0.003286
[500]	train-mae:0.003253
[501]	train-mae:0.00322
[502]	train-mae:0.003188
[503]	train-mae:0.003156
[504]	train-mae:0.003125
[505]	train-mae:0.003093
[506]	train-mae:0.003062
[507]	train-mae:0.003032
[508]	train-mae:0.003002
[509]	train-mae:0.002972
[510]	train-mae:0.002942
[511]	train-mae:0.002912
[512]	train-mae:0.002883
[513]	train-mae:0.002855
[514]	train-mae:0.002826
[515]	train-mae:0.002798
[516]	train-mae:0.00277
[517]	train-mae:0.002742
[518]	

[808]	train-mae:0.000332
[809]	train-mae:0.000331
[810]	train-mae:0.00033
[811]	train-mae:0.000329
[812]	train-mae:0.000329
[813]	train-mae:0.000328
[814]	train-mae:0.000327
[815]	train-mae:0.000326
[816]	train-mae:0.000325
[817]	train-mae:0.000324
[818]	train-mae:0.000324
[819]	train-mae:0.000323
[820]	train-mae:0.000322
[821]	train-mae:0.000321
[822]	train-mae:0.000321
[823]	train-mae:0.00032
[824]	train-mae:0.000319
[825]	train-mae:0.000319
[826]	train-mae:0.000318
[827]	train-mae:0.000318
[828]	train-mae:0.000317
[829]	train-mae:0.000316
[830]	train-mae:0.000316
[831]	train-mae:0.000315
[832]	train-mae:0.000314
[833]	train-mae:0.000313
[834]	train-mae:0.000313
[835]	train-mae:0.000312
[836]	train-mae:0.000312
[837]	train-mae:0.000311
[838]	train-mae:0.00031
[839]	train-mae:0.00031
[840]	train-mae:0.000309
[841]	train-mae:0.000309
[842]	train-mae:0.000308
[843]	train-mae:0.000308
[844]	train-mae:0.000307
[845]	train-mae:0.000306
[846]	train-mae:0.000305
[847]	train-mae:0.000305
[848

[1132]	train-mae:0.000218
[1133]	train-mae:0.000218
[1134]	train-mae:0.000218
[1135]	train-mae:0.000217
[1136]	train-mae:0.000217
[1137]	train-mae:0.000217
[1138]	train-mae:0.000217
[1139]	train-mae:0.000217
[1140]	train-mae:0.000216
[1141]	train-mae:0.000216
[1142]	train-mae:0.000216
[1143]	train-mae:0.000216
[1144]	train-mae:0.000216
[1145]	train-mae:0.000216
[1146]	train-mae:0.000215
[1147]	train-mae:0.000215
[1148]	train-mae:0.000215
[1149]	train-mae:0.000215
[1150]	train-mae:0.000215
[1151]	train-mae:0.000214
[1152]	train-mae:0.000214
[1153]	train-mae:0.000214
[1154]	train-mae:0.000214
[1155]	train-mae:0.000213
[1156]	train-mae:0.000213
[1157]	train-mae:0.000213
[1158]	train-mae:0.000213
[1159]	train-mae:0.000212
[1160]	train-mae:0.000212
[1161]	train-mae:0.000212
[1162]	train-mae:0.000212
[1163]	train-mae:0.000212
[1164]	train-mae:0.000211
[1165]	train-mae:0.000211
[1166]	train-mae:0.000211
[1167]	train-mae:0.000211
[1168]	train-mae:0.000211
[1169]	train-mae:0.000211
[1170]	train

[1449]	train-mae:0.000166
[1450]	train-mae:0.000166
[1451]	train-mae:0.000166
[1452]	train-mae:0.000166
[1453]	train-mae:0.000166
[1454]	train-mae:0.000165
[1455]	train-mae:0.000165
[1456]	train-mae:0.000165
[1457]	train-mae:0.000165
[1458]	train-mae:0.000165
[1459]	train-mae:0.000164
[1460]	train-mae:0.000164
[1461]	train-mae:0.000164
[1462]	train-mae:0.000164
[1463]	train-mae:0.000164
[1464]	train-mae:0.000164
[1465]	train-mae:0.000164
[1466]	train-mae:0.000163
[1467]	train-mae:0.000163
[1468]	train-mae:0.000163
[1469]	train-mae:0.000163
[1470]	train-mae:0.000163
[1471]	train-mae:0.000163
[1472]	train-mae:0.000163
[1473]	train-mae:0.000163
[1474]	train-mae:0.000162
[1475]	train-mae:0.000162
[1476]	train-mae:0.000162
[1477]	train-mae:0.000162
[1478]	train-mae:0.000162
[1479]	train-mae:0.000162
[1480]	train-mae:0.000161
[1481]	train-mae:0.000161
[1482]	train-mae:0.000161
[1483]	train-mae:0.000161
[1484]	train-mae:0.000161
[1485]	train-mae:0.000161
[1486]	train-mae:0.00016
[1487]	train-

In [12]:
import xgboost as xgb  #xgboost
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBRegressor  #xgb的sklearn包

xgbpredict = np.zeros((120000, 62))
    
for i in range(0,62):
    
    bst = XGBRegressor(
                     learning_rate =0.01,
                     n_estimators=5000,
                     max_depth=10,
                     min_child_weight=2,
                     silent = True,
                     nthread=4,
                     gamma=0,
                     reg_lambda=10,
                     reg_alpha = 10,
                     subsample=0.95,
                     colsample_bytree=0.95,
                     seed=27
                     )
 
    if i==60:
        name_of_column = 'Ret_PlusOne'
        y_train = training_csv[name_of_column]
        y_train_data = y_train.values
        
        bst.fit(x_train_data, y_train_data,eval_metric=WMAE)
        xgb_result_intraday = bst.predict(testing_data)
        xgbpredict[:,i]=xgb_result_intraday
        
    elif i==61:
        name_of_column = 'Ret_PlusTwo'
        y_train = training_csv[name_of_column]
        y_train_data = y_train.values
        
      
        
        bst.fit(x_train_data, y_train_data,eval_metric=WMAE)
        xgb_result_daily = bst.predict(testing_data)
        xgbpredict[:,i]=xgb_result_daily

    else:
        name_of_column = 'Ret_'+str(i+121)
        y_train = training_csv[name_of_column]
        y_train_data = y_train.values
        
        bst.fit(x_train_data, y_train_data,eval_metric=WMAE)
        xgb_result_daily = bst.predict(testing_data)
        xgbpredict[:,i]=xgb_result_daily

        

resultnum=np.zeros(7440000)
for j in range(0,120000):
    resultnum[62*j:62+62*j]=xgbpredict[j,:]
result_csv.iloc[0:,1]=resultnum
result_csv.to_csv('result-xgboost-med.csv',index=False)